In [1]:
# experiment.py

import numpy as np
import pandas as pd
import time
import logging
from data_generator import DataGenerator, StudyParameters, TestConfiguration, create_test_instance
from collections import defaultdict
from functools import lru_cache

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set a global random seed for reproducibility
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)

In [8]:
def dynamic_programming(instance):
    """
    Implement the Dynamic Programming (DP) method to compute the optimal solution.

    Args:
        instance (dict): Test instance containing parameters, booking classes, arrival probabilities,
                        reservation price parameters, and initial prices.

    Returns:
        float: Total expected revenue from the DP optimal solution.
    """
    params = instance['parameters']
    booking_classes = instance['booking_classes']
    arrival_probs = instance['arrival_probabilities']
    reservation_price_params = instance['reservation_price_params']
    initial_prices = instance['initial_prices']
    
    T = params.T
    N = params.N
    C = params.C
    price_min = params.price_min
    price_max = params.price_max
    
    # Precompute booking class details
    class_to_nights = {b: list(range(b[0], b[1]+1)) for b in booking_classes}
    class_length = {b: b[1] - b[0] +1 for b in booking_classes}
    
    @lru_cache(maxsize=None)
    def V(state, t):
        """
        Recursive value function.

        Args:
            state (tuple): Current state representing remaining capacities.
            t (int): Current time period.

        Returns:
            float: Maximum expected revenue from time t to T.
        """
        if t > T:
            return 0.0
        if all(x == 0 for x in state):
            return 0.0
        
        max_revenue = -np.inf
        best_price = None
        
        # Discrete price levels for efficiency
        price_levels = np.linspace(price_min, price_max, num=20)  # 5 discrete levels
        
        for p_avg in price_levels:
            expected_revenue = 0.0
            for b in booking_classes:
                pi_b = arrival_probs[t].get(b, 0.0)
                epsilon_b = reservation_price_params.get(b, 0.0)
                # Survival function F_bar = 1 - epsilon_b * p_avg
                F_bar = 1 - epsilon_b * p_avg
                F_bar = max(min(F_bar,1.0),0.0)  # Ensure within [0,1]
                purchase_prob = pi_b * F_bar
                feasible = all(state[day-1] >=1 for day in class_to_nights[b])
                if feasible:
                    revenue = class_length[b] * p_avg * purchase_prob
                    # Update state
                    new_state = list(state)
                    for day in class_to_nights[b]:
                        new_state[day-1] -=1
                    new_state = tuple(new_state)
                    # Recursive call
                    future_revenue = V(new_state, t+1)
                    expected_revenue += revenue + purchase_prob * future_revenue
            if expected_revenue > max_revenue:
                max_revenue = expected_revenue
                best_price = p_avg
        return max_revenue
    
    # Initial state: full capacity for each day
    initial_state = tuple([C]*N)
    
    total_revenue = V(initial_state, 1)
    logger.info(f"DP Optimal Total Expected Revenue: ${total_revenue:.2f}")
    return total_revenue

def stochastic_approximation(instance, num_samples=1000):
    """
    Implement the Stochastic Approximation Algorithm (SAA) method.

    Args:
        instance (dict): Test instance containing parameters, booking classes, arrival probabilities,
                        reservation price parameters, and initial prices.
        num_samples (int): Number of sample scenarios to generate.

    Returns:
        float: Total expected revenue from the SAA solution.
    """
    params = instance['parameters']
    booking_classes = instance['booking_classes']
    arrival_probs = instance['arrival_probabilities']
    reservation_price_params = instance['reservation_price_params']
    initial_prices = instance['initial_prices']
    
    T = params.T
    N = params.N
    C = params.C
    price_min = params.price_min
    price_max = params.price_max
    
    # Precompute booking class details
    class_to_nights = {b: list(range(b[0], b[1]+1)) for b in booking_classes}
    class_length = {b: b[1] - b[0] +1 for b in booking_classes}
    
    # Generate sample scenarios
    scenarios = []
    for sample_idx in range(num_samples):
        scenario = []
        state = [C] * N
        for t in range(1, T + 1):
            # Prepare the list of choices and their corresponding probabilities
            choices = booking_classes + [None]
            probabilities = [arrival_probs[t].get(b, 0.0) for b in booking_classes]
            probabilities.append(1 - sum(probabilities))
            
            # Convert to numpy array with dtype=object to handle mixed types
            choices_array = np.array(choices, dtype=object)
            probabilities_array = np.array(probabilities)
            
            # Normalize probabilities to ensure they sum to 1
            probabilities_array /= probabilities_array.sum()
            
            try:
                # Sample a booking class based on the probabilities
                booking_class = np.random.choice(choices_array, p=probabilities_array)
            except ValueError as ve:
                logger.error(f"Probability distribution at time {t} does not sum to 1. Sum: {probabilities_array.sum()}")
                raise ve
            
            if booking_class is not None:
                epsilon_b = reservation_price_params.get(booking_class, 0.0)
                # Offered average price
                p_avg = initial_prices[t].get(booking_class, price_min)
                
                # Compute survival function F_bar = 1 - epsilon_b * p_avg
                F_bar = 1 - epsilon_b * p_avg
                F_bar = max(min(F_bar,1.0),0.0)  # Ensure within [0,1]
                
                # Determine if the customer purchases based on F_bar
                purchase = np.random.rand() < F_bar
                
                # Check capacity constraints
                feasible = all(state[day - 1] >= 1 for day in class_to_nights[booking_class])
                
                if purchase and feasible:
                    # Accept booking
                    revenue = class_length[booking_class] * p_avg
                    scenario.append((booking_class, revenue))
                    for day in class_to_nights[booking_class]:
                        state[day - 1] -=1
                else:
                    # Reject booking
                    scenario.append((booking_class, 0.0))
            else:
                # No booking
                scenario.append((None, 0.0))
        scenarios.append(scenario)
    
    # Estimate expected revenue
    total_revenue = 0.0
    for scenario in scenarios:
        scenario_revenue = sum([revenue for _, revenue in scenario])
        total_revenue += scenario_revenue
    expected_revenue = total_revenue / num_samples
    logger.info(f"SAA Estimated Total Expected Revenue: ${expected_revenue:.2f}")
    return expected_revenue

In [9]:
def main():
    """
    Main function to set up and run the experiment comparing SAA and DP methods.
    """
    # Step 1: Setup for the Experiment
    # Define study parameters
    study_params = StudyParameters(
        T=10,  # Booking horizon
        N=5,   # Service horizon
        C=5,   # Capacity
        price_min=100.0,  # Example minimum price
        price_max=500.0,  # Example maximum price
        alpha=0.1,  # Smoothing parameter (unused in this basic implementation)
        beta=0.1    # Smoothing parameter (unused in this basic implementation)
    )
    
    # Initialize DataGenerator
    generator = DataGenerator(study_params, seed=GLOBAL_SEED)
    
    # Generate arrival probabilities
    arrival_probs = generator.generate_arrival_probabilities(demand_scenario='base')
    
    # Generate reservation price parameters
    reservation_price_params = generator.generate_reservation_price_params(market_condition='standard')
    
    # Generate initial prices using data_generator.py's function
    # Choose an initialization strategy, e.g., 'market_based'
    price_initialization = {
        'strategy': 'market_based',
        'params': None  # Add any strategy-specific parameters if needed
    }
    
    initial_prices = generator.generate_initial_prices(
        market_condition='standard',  # Match the market condition used for reservation price params
        initialization_strategy=price_initialization['strategy'],
        strategy_params=price_initialization['params']
    )
    
    # Create test instance
    test_instance = {
        'parameters': study_params,
        'booking_classes': generator.booking_classes,
        'arrival_probabilities': arrival_probs,
        'reservation_price_params': reservation_price_params,
        'initial_prices': initial_prices
    }
    
    # Step 2: Algorithm Implementation
    # Implement DP method
    start_time_dp = time.time()
    dp_revenue = dynamic_programming(test_instance)
    end_time_dp = time.time()
    dp_time = end_time_dp - start_time_dp
    
    # Implement SAA method
    start_time_saa = time.time()
    saa_revenue = stochastic_approximation(test_instance, num_samples=1000)
    end_time_saa = time.time()
    saa_time = end_time_saa - start_time_saa
    
    # Step 3: Evaluation and Output
    # Calculate gap
    gap = ((saa_revenue - dp_revenue) / dp_revenue) * 100 if dp_revenue !=0 else np.nan
    
    # Output results
    logger.info("\nExperiment Results")
    logger.info("===================")
    logger.info(f"Dynamic Programming (DP) Revenue: ${dp_revenue:.2f}")
    logger.info(f"Stochastic Approximation Algorithm (SAA) Revenue: ${saa_revenue:.2f}")
    logger.info(f"Revenue Gap (SAA - DP) / DP * 100%: {gap:.2f}%")
    logger.info(f"DP Computation Time: {dp_time:.4f} seconds")
    logger.info(f"SAA Computation Time: {saa_time:.4f} seconds")
    
    # Step 4: Documentation and Explanation
    # Summary in Markdown
    summary = f"""
# Experiment Summary

## Instance Setup
- **Booking Horizon (T):** {study_params.T} periods
- **Service Horizon (N):** {study_params.N} days
- **Room Capacity (C):** {study_params.C} rooms
- **Price Range:** ${study_params.price_min} - ${study_params.price_max}
- **Demand Scenario:** Base
- **Market Condition:** Standard
- **Price Initialization Strategy:** {price_initialization['strategy']}

## Revenue Comparison
- **Dynamic Programming (DP) Revenue:** ${dp_revenue:.2f}
- **Stochastic Approximation Algorithm (SAA) Revenue:** ${saa_revenue:.2f}
- **Revenue Gap:** {gap:.2f}%

## Computation Times
- **DP Computation Time:** {dp_time:.4f} seconds
- **SAA Computation Time:** {saa_time:.4f} seconds

## Key Takeaways
- The SAA method **{'approaches' if abs(gap) < 5 else 'significantly deviates from'}** the optimal DP solution in terms of revenue performance.
- The SAA method **{'is faster' if saa_time < dp_time else 'is slower'}** compared to the DP method.
- **Future Work:** Expand the experiment to larger and more complex instances to assess scalability and robustness.

    """
    # Print summary
    print(summary)

In [10]:
if __name__ == "__main__":
    main()

INFO:data_generator:Generated 15 booking classes with max LOS = 7
INFO:data_generator:Initialized DataGenerator with 15 booking classes
INFO:data_generator:
Demand Scenario: base
INFO:data_generator:Average daily arrival probability: 0.950
INFO:data_generator:Maximum daily arrival probability: 0.950
INFO:data_generator:Minimum daily arrival probability: 0.950
INFO:data_generator:
Average arrival probabilities by day of week:
INFO:data_generator:Sunday: 0.075
INFO:data_generator:Monday: 0.056
INFO:data_generator:Tuesday: 0.057
INFO:data_generator:Wednesday: 0.057
INFO:data_generator:Thursday: 0.065
INFO:data_generator:Friday: 0.000
INFO:data_generator:Saturday: 0.000
INFO:data_generator:
Reservation Price Parameters for STANDARD Market:
INFO:data_generator:Market Characteristics:
INFO:data_generator:- Base Price Sensitivity: 0.50
INFO:data_generator:- Length of Stay Factor: 0.100
INFO:data_generator:- Weekend Premium: 0.20
INFO:data_generator:- Random Variation: ±0.10
INFO:data_generato


# Experiment Summary

## Instance Setup
- **Booking Horizon (T):** 10 periods
- **Service Horizon (N):** 5 days
- **Room Capacity (C):** 5 rooms
- **Price Range:** $100.0 - $500.0
- **Demand Scenario:** Base
- **Market Condition:** Standard
- **Price Initialization Strategy:** market_based

## Revenue Comparison
- **Dynamic Programming (DP) Revenue:** $739.11
- **Stochastic Approximation Algorithm (SAA) Revenue:** $1516.70
- **Revenue Gap:** 105.21%

## Computation Times
- **DP Computation Time:** 7.4338 seconds
- **SAA Computation Time:** 0.1359 seconds

## Key Takeaways
- The SAA method **significantly deviates from** the optimal DP solution in terms of revenue performance.
- The SAA method **is faster** compared to the DP method.
- **Future Work:** Expand the experiment to larger and more complex instances to assess scalability and robustness.

    
